# Making a Parametric Stellar Population

The core element of the parametric modelling in `Synthesizer` is the `parametric.Stars` object. This contains everything necessary to describe a stellar population including:

- A 2D SFZH grid containing the stellar mass formed in stellar age and metallicity bins.

- A `Morphology` object describing it's distribution (optional but required for imaging).

- The axes of the SFZH grid.

- The total initial stellar mass.

And if calculated by the user (see the parametric `Sed` [docs](generate_sed.ipynb)):

- The stellar spectra.

- Line luminosities.

There are a number of different methods to define a parametric stellar population (from here on a `Stars` object). In what follows we will focus on user defined values/arrays but note that each different method could instead use simulated data (e.g. from Semi Analytic Models) rather than user defined values.

The possible ways of defining a `Stars` object are (the age and metallicity axis of the SFZH must always be supplied):

- Explictly providing the SFZH grid itself. Note that this will mean all other definitions below are ignored.

- Providing singular age and metallicity values to define an "instantaneous" SFZH, i.e. one where all the stellar mass is in a single age and metallicity bin.

- Providing arrays which contain the SFH and metallicity distribution in terms of stellar mass formed.

- Providing SFH and metallcity distribution functions (from `parametric.sf_hist` and `parametric.metal_dist`). These will be used internally to compute the arrays from the previous point.

- Any combination of the above.

Below we simply import some packages and objects we'll need.

In [ ]:
import numpy as np
from synthesizer.parametric import SFH, Stars, ZDist
from unyt import Myr

## Defining an instantaneous SFZH

To define an instantaneous SFZH we need only to provide the age and metallicity of the SFZH bin, alongside the SFZH axis arrays and the total initial mass of the `Stars` object. Note that the `log10ages` and `log10metallicites`/`metallicities` will likely come from a `Grid` object in most workflows (see the `Grid` [docs](../grids/grids_example.ipynb)).

In [ ]:
# Create the SFZH axis arrays
log10ages = np.arange(6.0, 10.5, 0.1)  # log10(age/yr)
log10metallicities = np.arange(-5.0, -1.5, 0.25)
metallicities = 10**log10metallicities

# Get the instantaneous Stars
inst_stars = Stars(
    log10ages,
    metallicities,
    sf_hist=100 * Myr,
    metal_dist=10**-3,
    initial_mass=10**9,
)

To see the created SFZH you can print some basic statistics by printing the `Stars` object.

In [ ]:
print(inst_stars)

Or to probe the SFZH you can plot it using the provided helper function.

In [ ]:
fig, ax = inst_stars.plot_sfzh(show=True)

## Defining SFZH from arrays

Of course, most of the time you won't want an instantaneous SFZH but instead a distribution. One way to achieve this is to pass arrays describing the SFH and metallicity distribution. This is particularly useful if you have data explictly describing the ages and metallicities of a stellar population. However, here we'll demonstrate this with entirely unphysical arrays.

In [ ]:
# Create arrays for SFH and ZDist
sf_hist = np.zeros_like(log10ages)
metal_dist = np.zeros_like(metallicities)
sf_hist[10:15] = 1
sf_hist[15:20] = 0.5
metal_dist[5:6] = 1
metal_dist[6:8] = 0.5

# Get the Stars
arr_stars = Stars(
    log10ages,
    metallicities,
    sf_hist=sf_hist,
    metal_dist=metal_dist,
    initial_mass=10**9,
)
print(arr_stars)
fig, ax = arr_stars.plot_sfzh(show=True)

## Defining SFZH using functions

You won't always have explict arrays to describe the age and metallicity distributions. For these situations `Synthesizer` provides a suite of functions to describe the SFH and metallicity distribution. 

SFH parametrisations can be found in the `SFH` module, while metallicity distribution parametrisations can be found in the `ZDist` module. To see a full list of available parametrisations just print the `parametrisations` list from these modules`.

In [ ]:
print(SFH.parametrisations)
print(ZDist.parametrisations)

To use one of these parametrisations you have to first define an instance of a parametrisation and then pass it to the `Stars` object at instantiation. Below we demonstrate a few examples using different parametrisations.

### Constant SFH and metallicity delta function

In [ ]:
# Define a delta function for metallicity
metal_dist = ZDist.DeltaConstant(log10metallicity=-2.5)
print(metal_dist)

# Define a constant SFH
sfh = SFH.Constant(duration=100 * Myr)
print(sfh)

# Create the Stars object
const_stars = Stars(
    log10ages,
    metallicities,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=10**9,
)
print(const_stars)
fig, ax = const_stars.plot_sfzh(show=True)

### Truncated exponential SFH and normal metallicity

In [ ]:
# Define a delta function for metallicity
Z_p = {"Z": 0.01}
metal_dist = ZDist.Normal(mean=0.0005, sigma=0.001)
print(metal_dist)

# Define an exponential SFH
sfh = SFH.TruncatedExponential(tau=100 * Myr, max_age=200 * Myr)
print(sfh)

# Create the Stars object
exp_stars = Stars(
    log10ages,
    metallicities,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=10**9.5,
)
print(exp_stars)
fig, ax = exp_stars.plot_sfzh(show=True)

### Log normal SFH and instantaneous metallicity

Remember we can combine any type of SFH and metallicity distribution inputs. Here we'll use a SFH function and an instantaneous metallicity (analogus to a delta function).

In [ ]:
# Define an exponential SFH
sfh = SFH.LogNormal(tau=10, max_age=500 * Myr, peak_age=100 * Myr)
print(sfh)

# Create the Stars object
logn_stars = Stars(
    log10ages,
    metallicities,
    sf_hist=sfh,
    metal_dist=0.005,
    initial_mass=10**9,
)
print(logn_stars)
fig, ax = logn_stars.plot_sfzh(show=True)

### Defining your own parametrisations

If `Synthesizer` doesn't already include a parametrisation you need, you can add a custom parametrisation by defining a new class.

The important requirements of a custom functions are:

- They inherit the `Common` parent class.

- They instantiate the parent class (`SFH/ZDist.Common`) with the model parameters and a meaningful name.

- They implement a private `_sfr` method which takes a single age and returns a single SFR weight in the `SFH` case, or they implement a `_weight` method which take a single metallicity and return a single metallicity weight in the `ZDist` case.

- The `SFH` methods should perform their calculations in years.

Below we demonstrate how to define a custom function using existing SFH and metallicity distribution functions for demonstration purposes.

In [ ]:
class Normal(ZDist.Common):
    """
    A normally distributed metallicity distribution.

    Attributes:
        mean (float)
            The mean of the normal distribution.
        sigma (float)
            The standard deviation of the normal distribution.
    """

    def __init__(self, mean, sigma):
        """
        Initialise the metallicity distribution and parent.

        Either metallicity or log10metallicity must be provided.

        Args:
            mean (float)
                The mean of the normal distribution.
            sigma (float)
                The standard deviation of the normal distribution.
        """

        # Instantiate the parent
        ZDist.Common.__init__(
            self,
            name="Normal",
            mean=mean,
            sigma=sigma,
        )

        # Define this models parameters
        self.mean = mean
        self.sigma = sigma

    def _weight(self, metal):
        """
        Return the distribution at a metallicity.

        Args:
            metal (float)
                The (linear) metallicity at which to evaluate the distribution.

        Returns:
            float
                The weight of the metallicity distribution at metal.
        """
        norm = 1 / (self.sigma * np.sqrt(2 * np.pi))
        exponent = ((metal - self.mean) / self.sigma) ** 2
        return norm * np.exp(-0.5 * exponent)


class Constant(SFH.Common):
    """
    A constant star formation history.

    The SFR is defined such that:
        sfr = 1; t<=duration
        sfr = 0; t>duration

    Attributes:
        duration (float)
            The duration of the period of constant star formation.
    """

    def __init__(self, duration):
        """
        Initialise the parent and this parametrisation of the SFH.

        Args:
            duration (unyt_quantity)
                The duration of the period of constant star formation.
        """

        # Initialise the parent
        SFH.Common.__init__(self, name="Constant", duration=duration)

        # Set the model parameters
        self.duration = duration.to("yr").value

    def _sfr(self, age):
        """
        Get the amount SFR weight in a single age bin.

        Args:
            age (float)
                The age (in years) at which to evaluate the SFR.
        """

        # Set the SFR based on the duration.
        if age <= self.duration:
            return 1.0
        return 0.0


# Define a scaled delta function for metallicity
metal_dist = Normal(mean=0.05, sigma=0.001)
print(metal_dist)

# Define an exponential SFH
sfh = Constant(duration=500 * Myr)
print(sfh)

# Create the Stars object
custom_stars = Stars(
    log10ages,
    metallicities,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=10**9,
)
print(custom_stars)
fig, ax = custom_stars.plot_sfzh(show=True)

## Combining `Stars`

We can also combine individual `Stars` objects together to produce more complicated star formation and metal enrichment histories by simply adding them.

In [ ]:
combined = arr_stars + const_stars + exp_stars + logn_stars + custom_stars
print(combined)
fig, ax = combined.plot_sfzh()